In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Carregar o modelo

In [6]:
im = cv2.imread("./teste.jpg")
cv2.imshow("win", im)
cv2.waitKey(0)

# Close the window after the key press
cv2.destroyAllWindows()

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

[12/09 12:39:33 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


/home/ricardofachini/miniconda3/envs/visao-computacional/lib/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647380992/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("w", out.get_image()[:, :, ::-1])

cv2.waitKey(0)

# Close the window after the key press
cv2.destroyAllWindows()

Carregar dataset de car-damage

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# !curl -L "https://universe.roboflow.com/ds/5sncDNZAeT?key=lgL2BuIUSz" > roboflow.zip; unzip roboflow.zip -d datasets/car-damage-s4; rm roboflow.zip
# !curl -L "https://universe.roboflow.com/ds/R0WssxzPHH?key=DrYHDd0r3w" > roboflow.zip; unzip roboflow.zip -d datasets/severity; rm roboflow.zip


# Replace with paths to your dataset
register_coco_instances("car-damage-v8-train", {}, "datasets/car-damage-v8/train/_annotations.coco.json", "datasets/car-damage-v8/train")
register_coco_instances("car-damage-v8-test", {}, "datasets/car-damage-v8/valid/_annotations.coco.json", "datasets/car-damage-v8/valid")
# register_coco_instances("car-damage-s4-train", {}, "datasets/car-damage-s4/train/_annotations.coco.json", "datasets/car-damage-s4/train")
# register_coco_instances("car-damage-s4-test", {}, "datasets/car-damage-s4/test/_annotations.coco.json", "datasets/car-damage-s4/test")
# register_coco_instances("severity-train", {}, "datasets/severity/train/_annotations.coco.json", "datasets/severity/train")
# register_coco_instances("severity-test", {}, "datasets/severity/test/_annotations.coco.json", "datasets/severity/test")
# register_coco_instances("severity-val", {}, "datasets/severity/valid/_annotations.coco.json", "datasets/severity/valid")

# Verify if the dataset is correctly registered
my_dataset_metadata = MetadataCatalog.get("car-damage-v8-train")
dataset_dicts = DatasetCatalog.get("car-damage-v8-train")

print(len(my_dataset_metadata.thing_classes))



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



2


Printar resultados

In [3]:
''
from detectron2.utils.visualizer import ColorMode

import random
import cv2
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

# Get the dataset metadata and dataset dicts
# dataset_dicts = DatasetCatalog.get("severity-test")
# metadata = MetadataCatalog.get("severity-test")

# Plot a few random images from the dataset
for d in random.sample(dataset_dicts, 3):  # Change the number to print more or fewer images
    img = cv2.imread(d["file_name"])

    # Create a visualizer and draw annotations (bounding boxes, etc.)
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_metadata, scale=0.8)
    vis = visualizer.draw_dataset_dict(d)

    # Display the image with matplotlib
    plt.figure(figsize=(12, 8))
    cv2.imshow("w", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    # Close the window after the key press
    cv2.destroyAllWindows()


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

# Selecionar modelo e treinar dataset de treino

In [7]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) #
cfg.DATASETS.TRAIN = ("car-damage-v8-train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300 #2000
cfg.SOLVER.STEPS = (3000, 4000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[12/09 12:39:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[12/09 12:39:38 d2.engine.train_loop]: Starting training from iteration 0
[12/09 12:39:44 d2.utils.events]:  eta: 0:01:16  iter: 19  total_loss: 2.243  loss_cls: 1.153  loss_box_reg: 0.3044  loss_mask: 0.6931  loss_rpn_cls: 0.05411  loss_rpn_loc: 0.01664    time: 0.2763  last_time: 0.2565  data_time: 0.0083  last_data_time: 0.0023   lr: 1.6068e-05  max_mem: 1940M
[12/09 12:39:50 d2.utils.events]:  eta: 0:01:09  iter: 39  total_loss: 1.963  loss_cls: 0.7775  loss_box_reg: 0.3288  loss_mask: 0.6883  loss_rpn_cls: 0.1012  loss_rpn_loc: 0.02477    time: 0.2726  last_time: 0.2290  data_time: 0.0024  last_data_time: 0.0023   lr: 3.2718e-05  max_mem: 1940M
[12/09 12:39:55 d2.utils.events]:  eta: 0:01:04  iter: 59  total_loss: 1.493  loss_cls: 0.4476  loss_box_reg: 0.2764  loss_mask: 0.6786  loss_rpn_cls: 0.07568  loss_rpn_loc: 0.01787    time: 0.2761  last_time: 0.2990  data_time: 0.0026  last_data_time: 0.0028   lr: 4.9367e-05  max_mem: 1940M
[12/09 12:40:01 d2.utils.events]:  eta: 0:01:00  

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

Evaluate

In [8]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
import os

# Configurações para o modelo treinado
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Caminho para o modelo salvo
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Ajuste o threshold de detecção, se necessário
predictor = DefaultPredictor(cfg)


[12/09 12:41:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


/home/ricardofachini/miniconda3/envs/visao-computacional/lib/python3.9/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return to

Visualizar resultado

In [9]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=my_dataset_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    print(outputs["instances"].pred_classes)  # Classes preditas
    print(outputs["instances"].pred_boxes)
    cv2.imshow("w", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)

    # Close the window after the key press
    cv2.destroyAllWindows()


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
Boxes(tensor([[106.1708,  19.2041, 348.0602, 191.4880],
        [157.7554,  52.6061, 304.0989, 248.8642],
        [ 46.7163,   1.3430, 370.5620, 322.3849],
        [ 56.7064,  75.1521, 175.7081, 250.3241],
        [168.0660,  33.1937, 314.6277, 171.0528],
        [ 51.6978,  52.9554, 268.5035, 231.1189],
        [249.8474, 215.3847, 314.5869, 312.8590],
        [ 84.1882,  96.1413, 154.4855, 224.4031],
        [ 30.4920,  86.2998, 147.8165, 291.0134],
        [ 60.8694,  29.2553, 158.2602, 137.3021],
        [  5.9359,  33.0697, 191.3265, 308.7455]], device='cuda:0'))
tensor([1], device='cuda:0')
Boxes(tensor([[164.1096,  62.0236, 411.2791, 220.0856]], device='cuda:0'))
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
Boxes(tensor([[177.7712,  76.8734, 331.8645, 363.7521],
        [184.2011, 194.4961, 282.4620, 361.8541],
        [191.3517,  28.5196, 315.0508, 251.7374],
        [ 13.3866,  14.7259, 375.9769, 396.6322],


Testar uma imagem qualquer